# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

In [ ]:
#python -m venv wikipedia_scraper_env
.\wikipedia_scraper_env\Scripts\Activate.ps1

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [ ]:
# import the requests library (1 line)
import requests
import json
from bs4 import BeautifulSoup
import re
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = "/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{status_url}")
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print("OK")
else:
    print(req.reason)

OK


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [9]:
# Set the countries_url variable (1 line)
countries_url = "/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{countries_url}")
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [19]:
# Set the cookie_url variable (1 line)
cookie_url = "/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(root_url+"/"+cookie_url).cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=40d6dc7c-0c54-45ec-842e-db10f9332a09 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [ ]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
# display the countries variable (1 line)
print(countries)

['us', 'be', 'ma', 'ru', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [20]:
# Set the leaders_url variable (1 line)
leaders_url = "/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies).json()
# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [25]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
cookie_url = "/cookie"
cookies = requests.get(root_url+"/"+cookie_url).cookies
leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':'be'}).json()
# display the leaders variable (1 line)
print(json.dumps(leaders, indent=4,separators=(". ", " = ")))

[
    {
        "id" = "Q12978". 
        "first_name" = "Guy". 
        "last_name" = "Verhofstadt". 
        "birth_date" = "1953-04-11". 
        "death_date" = null. 
        "place_of_birth" = "Dendermonde". 
        "wikipedia_url" = "https://nl.wikipedia.org/wiki/Guy_Verhofstadt". 
        "start_mandate" = "1999-07-12". 
        "end_mandate" = "2008-03-20"
    }. 
    {
        "id" = "Q12981". 
        "first_name" = "Yves". 
        "last_name" = "Leterme". 
        "birth_date" = "1960-10-06". 
        "death_date" = null. 
        "place_of_birth" = "Wervik". 
        "wikipedia_url" = "https://nl.wikipedia.org/wiki/Yves_Leterme". 
        "start_mandate" = "2009-11-25". 
        "end_mandate" = "2011-12-06"
    }. 
    {
        "id" = "Q12983". 
        "first_name" = "Herman". 
        "last_name" = "None". 
        "birth_date" = "1947-10-31". 
        "death_date" = null. 
        "place_of_birth" = "Etterbeek". 
        "wikipedia_url" = "https://nl.wikipedia.org/wik

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leaders_per_country = {}
for country in countries:
    leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json()
    leaders_per_country[country] = leaders

#print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

In [ ]:
# or 1 line
leaders_per_country = {country:requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json()
  for country in countries}

#print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [29]:
# < 15 lines
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(root_url+"/cookie").cookies
    countries_url = "/countries"
    countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    leaders_per_country = {country:requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json() 
                           for country in countries}
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [31]:
# 2 lines
leaders_per_country = get_leaders()
for country,leaders in leaders_per_country.items():
    print(country , leaders)

us [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [113]:
# 3 lines
one_leader = "https://en.wikipedia.org/wiki/Barack_Obama"
req = requests.get(one_leader)
#print(req.text)

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [114]:
# 3 lines
soup = BeautifulSoup(req.text, "html")
#print(soup.prettify())

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [112]:
# 2 lines
one_leader = 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86'
req = requests.get(one_leader)
soup = BeautifulSoup(req.text, "html")
paragraphs = [para.text.strip() for para in soup.find_all("p") if len(para.text.strip())>0]
paragraphs

['هذه نسخة متحقق منها من هذه الصفحة',
 'مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.',
 'ولد في الرباط ليكون أكبر الأبناء الذكور للملك الحسن الثاني من زوجته لطيفة حمو، لذا فقد كان وريثًا واضحًا للعرش ووليًّا للعهد منذ ولادته كما ينص الدستور، بدأ تعليمه في سن الرابعة بالكُتَّاب القرآني الملحق بالقصر الملكي بينما أكمله بالخارج إلى أن نال شهادة الدكتوراه في الحقوق مع مرتبة الشرف من جامعة (نيس صوفيا آنتيبوليس) الفرنسية.',
 'الملك محمد السادس هو أحد أفراد السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، والتي تحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين،[12] وهو بذلك يُعتبر من أقدم الزعماء العرب الأحياء في الحكم، يسبقه في الأقدمية الملك عبدالله الثاني بن الحسين ملك الأردن والملك حمد بن عيسى آل خليفة 

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [ ]:
# <10 lines
import re
for para in paragraphs:
    if(re.match('.*\\[1\\].*',para)):
        first_paragraph = para
        break

Дми́трий Анато́льевич Медве́дев (род. 14 сентября 1965[1][2][…], Ленинград, СССР[3]) — российский государственный и политический деятель. Первый заместитель председателя Военно-промышленной комиссии Российской Федерации с 26 декабря 2022 года. Заместитель председателя Совета Безопасности Российской Федерации с 16 января 2020 года. Председатель партии «Единая Россия» с 26 мая 2012 года[4][5].


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [150]:
# 10 lines
def get_first_paragraph_1arg(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.text, "html")
    paragraphs = [para.text.strip() for para in soup.find_all("p") if len(para.text.strip())>0]
    for para in paragraphs:
        if((re.match('.*[1-9].*',para)) and (len(para.split()) > 20)):
            first_paragraph = para
            break
    return first_paragraph

In [151]:
# Test: 3 lines
#wiki_url = "https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87"
#wiki_url = "https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande"
#wiki_url = "https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86"
wiki_url = "https://en.wikipedia.org/wiki/Gerald_Ford"
print(get_first_paragraph_1arg(wiki_url))

https://en.wikipedia.org/wiki/Gerald_Ford
Gerald Rudolph Ford Jr.[a] (born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was the 38th president of the United States, serving from 1974 to 1977. A member of the Republican Party, Ford assumed the presidency after President Richard Nixon resigned, under whom he had served as the 40th vice president from 1973 to 1974. Prior to that, he served as a member of the United States House of Representatives from 1949 to 1973.


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [157]:
# 3 lines
first_para = get_first_paragraph_1arg(wiki_url)
#print(first_para)
print(re.sub("\\[.{,2}\\]","",first_para))

https://en.wikipedia.org/wiki/Gerald_Ford
Gerald Rudolph Ford Jr. (born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was the 38th president of the United States, serving from 1974 to 1977. A member of the Republican Party, Ford assumed the presidency after President Richard Nixon resigned, under whom he had served as the 40th vice president from 1973 to 1974. Prior to that, he served as a member of the United States House of Representatives from 1949 to 1973.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [125]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.text, "html")
    paragraphs = [para.text.strip() for para in soup.find_all("p") if len(para.text.strip())>0]
    for para in paragraphs:
        if(re.match('.*[1-9].*',para)):
            first_paragraph = re.sub("\\[.{,2}\\]","",para)
            break
    return first_paragraph

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [126]:
# < 20 lines
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(root_url+"/cookie").cookies
    countries_url = "/countries"
    leaders_url = "/leaders"
    countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    leaders_per_country = {country:requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json() 
                           for country in countries}
    for country,leaders in leaders_per_country.items():
        for each in leaders: 
            each["first_wiki_para"] = get_first_paragraph(each["wikipedia_url"])
    return leaders_per_country

In [127]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
for country,leaders in leaders_per_country.items():
    print(country , leaders)

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [177]:
# < 25 lines
class PageNotFound(Exception):
    pass

def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(root_url+"/cookie").cookies
    countries_url = "/countries"
    leaders_url = "/leaders"
    countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    leaders_per_country = {}
  
    for country in countries:
        try:
            leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json()
            leaders_per_country[country] = leaders
        except:
            cookies = requests.get(root_url+"/cookie").cookies
            leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json()
            leaders_per_country[country] = leaders
        
    for country,leaders in leaders_per_country.items():
        for each in leaders: 
            each["first_wiki_para"] = get_first_paragraph(each["wikipedia_url"])
    return leaders_per_country

Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [153]:
# < 20 lines
from requests import Session
def get_first_paragraph(wikipedia_url,session):
    print(wikipedia_url) # keep this for the rest of the notebook
    req = session.get(wikipedia_url)
    soup = BeautifulSoup(req.text, "html")
    paragraphs = [para.text.strip() for para in soup.find_all("p") if len(para.text.strip())>0]
    for para in paragraphs:
        if((re.match('.*[1-9].*',para)) and (len(para.split()) > 20)):
            first_paragraph = re.sub("\\[.{,2}\\]","",para)
            break
    return first_paragraph

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [154]:
# <25 lines
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(root_url+"/cookie").cookies
    countries_url = "/countries"
    leaders_url = "/leaders"
    countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    leaders_per_country = {country:requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={'country':country}).json() 
                           for country in countries}
    with Session() as session:
        for country,leaders in leaders_per_country.items():
            for each in leaders: 
                each["first_wiki_para"] = get_first_paragraph(each["wikipedia_url"], session)
    return leaders_per_country


Test your new functions.



In [155]:
leaders_per_country = get_leaders()
for country,leaders in leaders_per_country.items():
    print(country , leaders)

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [173]:
# 3 lines
with open("leaders.json","w") as file:
    json.dump(leaders_per_country, file, indent=4) #, separators=(". ", " = "))

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [174]:
# 3 lines
with open("leaders.json","r") as file:
    leader_info = json.load(file)
print(leader_info["us"][0])

{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_wiki_para': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".'}


Make a function `save(leaders_per_country)` to call this code easily.

In [175]:
# 3 lines
def save(leaders_per_country,filename):
    with open(filename,"w") as file:
        json.dump(leaders_per_country, file, indent=4, separators=(". ", " = "))

In [176]:
# Call the function (1 line)
save(leaders_per_country,"leaders.json")

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!